In [189]:
import sqlite3
import os
import pandas as pd
import numpy as np

## ======== CREATE DataFrames ========

In [190]:
#  df_Ar    ==> DataFrame 'article'
#  df_Au    ==> DataFrame 'author'
#  df_Ar_Au ==> DataFrame 'article_author'

In [191]:
# ===================================
#   CREATE DataFrame 'article'
# Read csv file with the articles
# ===================================

In [192]:
# Read csv to DataFrame
df_Ar = pd.read_csv ('./nyt_csv_files/nyt_data_raw.csv', low_memory=False)
df_Ar.head(3)

,_id,abstract,web_url,snippet,lead_paragraph,print_section,print_page,pub_date,document_type,news_desk,section_name,type_of_material,word_count,headline_main,headline_print_headline,byline_original,byline_organization
0,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,Our president as a terrible toddler.,https://www.nytimes.com/2020/09/30/opinion/pre...,Our president as a terrible toddler.,When the nation looks back on the presidential...,A,27,2020-10-01T00:05:51+0000,article,OpEd,Opinion,Op-Ed,925,Anyone Else Want to See Trump ‘Shut Up’?,Anyone Else Want Trump To ‘Shut Up’?,By Gail Collins,NaN
1,nyt://article/55c14af4-0542-53ef-80aa-65e2440e...,The Lakers fell into a deficit early but fough...,https://www.nytimes.com/2020/09/30/sports/bask...,The Lakers fell into a deficit early but fough...,LeBron James is not accustomed to Game 1s like...,NaN,NaN,2020-10-01T00:30:45+0000,article,Sports,Sports,News,1723,How the Lakers Beat the Heat in Game 1 of the ...,NaN,By Sopan Deb and Marc Stein,NaN
2,nyt://article/9a7ef9e0-1334-56b2-a7f1-288c4887...,"Instead of condemning violent groups, the pres...",https://www.nytimes.com/2020/09/30/opinion/tru...,"Instead of condemning violent groups, the pres...",President Trump didn’t hurt Joe Biden in Tuesd...,A,27,2020-10-01T00:43:28+0000,article,OpEd,Opinion,Op-Ed,902,Trump Calls on Extremists to ‘Stand By’,Trump Calls On Extremists To ‘Stand By’,By Nicholas Kristof,NaN


In [193]:
# Separate date and time. 
# Create two new columns
df_Ar['a_date'] = pd.to_datetime(df_Ar['pub_date']).dt.date
df_Ar['a_time'] = pd.to_datetime(df_Ar['pub_date']).dt.time

In [194]:
# Create a new column of authors 
# separated by comma
# and CLEAN the authors column

# '^By.*Ezra Klein Show.*$': 'The Ezra Klein Show',  
    
dic = { # Strings at the start 
        'By '              : '',
        'By ‘'             : '',
        'Show.$'           : 'Show',     # I does not work. Why???     
        'Text by '         : '',
        'Video by '        : '',
        'Videos by '       : '',
        'Written by '      : '', 
        'Compiled by '     : '',
        'Artwork by '      : '',
        'Produced by '     : '',
        'Reporting by '    : '',
        'Selected by '     : '',
        'Photographs by '  : '',
        'Interviews by '   : '',
        'Interviews by '   : '',
        'Introduction by ' : '',
        'Photo Essay by '  : '',
        'Illustrations by ': '',
        'Photographs, Text by ': '',
       
        # Strings in the middle
        ' Text by '        : '',
        ' Video by '       : '',
        ' posters by '     : '',
        ' Flowers by '     : '',
        ' photographs by ' : '',

        # Strings in the middle that need to be replaced with ,
        '\;'                      : ',',
        ' and '                  : ', ',
        ' with Text by '         : ', ',
        ' with Photographs by '  : ', ',
       
        # Strings at the end
        ' for The New York Times': '',
        ' For The New York Times': ''
      }

#dic = {r"\b{}\b".format(k): v for k, v in dic.items()}

df_Ar['authors'] = df_Ar['byline_original'].replace(dic, regex=True)
df_Ar.head()

,_id,abstract,web_url,snippet,lead_paragraph,print_section,print_page,pub_date,document_type,news_desk,section_name,type_of_material,word_count,headline_main,headline_print_headline,byline_original,byline_organization,a_date,a_time,authors
0,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,Our president as a terrible toddler.,https://www.nytimes.com/2020/09/30/opinion/pre...,Our president as a terrible toddler.,When the nation looks back on the presidential...,A,27,2020-10-01T00:05:51+0000,article,OpEd,Opinion,Op-Ed,925,Anyone Else Want to See Trump ‘Shut Up’?,Anyone Else Want Trump To ‘Shut Up’?,By Gail Collins,NaN,2020-10-01,00:05:51,Gail Collins
1,nyt://article/55c14af4-0542-53ef-80aa-65e2440e...,The Lakers fell into a deficit early but fough...,https://www.nytimes.com/2020/09/30/sports/bask...,The Lakers fell into a deficit early but fough...,LeBron James is not accustomed to Game 1s like...,NaN,NaN,2020-10-01T00:30:45+0000,article,Sports,Sports,News,1723,How the Lakers Beat the Heat in Game 1 of the ...,NaN,By Sopan Deb and Marc Stein,NaN,2020-10-01,00:30:45,"Sopan Deb, Marc Stein"
2,nyt://article/9a7ef9e0-1334-56b2-a7f1-288c4887...,"Instead of condemning violent groups, the pres...",https://www.nytimes.com/2020/09/30/opinion/tru...,"Instead of condemning violent groups, the pres...",President Trump didn’t hurt Joe Biden in Tuesd...,A,27,2020-10-01T00:43:28+0000,article,OpEd,Opinion,Op-Ed,902,Trump Calls on Extremists to ‘Stand By’,Trump Calls On Extremists To ‘Stand By’,By Nicholas Kristof,NaN,2020-10-01,00:43:28,Nicholas Kristof
3,nyt://article/ecb7bde1-76af-5954-8cf3-9ad84b15...,Facebook on Wednesday said it would prohibit t...,https://www.nytimes.com/2020/09/30/us/election...,NaN,Facebook on Wednesday said it would prohibit t...,NaN,NaN,2020-10-01T00:45:04+0000,article,Politics,U.S.,News,350,Facebook will forbid ads that undermine the le...,NaN,By Mike Isaac,NaN,2020-10-01,00:45:04,Mike Isaac
4,nyt://article/4bb2b763-0088-5e10-b204-19e404f7...,If the Democratic Party claims to value Black ...,https://www.nytimes.com/2020/09/30/opinion/mik...,If the Democratic Party claims to value Black ...,"At the end of the Civil War in 1865, Mississip...",A,22,2020-10-01T00:45:17+0000,article,OpEd,Opinion,Op-Ed,1412,"Can Mike Espy Make History, Again?","Can Mike Espy Make History, Again?",By Charles M. Blow,NaN,2020-10-01,00:45:17,Charles M. Blow


In [195]:
# Create a row index. from 0 to ...

# Reset index, BUT save old
df_Ar = df_Ar.reset_index()

# Rename columns
df_Ar.rename(columns={'index': 'article_id', '_id': 'original_id'}, inplace=True)

# Write DataFrame to csv file
df_Ar.to_csv('./nyt_csv_files/nyt_data_test.csv', index=False)

df_Ar.head(3)

,article_id,original_id,abstract,web_url,snippet,lead_paragraph,print_section,print_page,pub_date,document_type,...,section_name,type_of_material,word_count,headline_main,headline_print_headline,byline_original,byline_organization,a_date,a_time,authors
0,0,nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d33...,Our president as a terrible toddler.,https://www.nytimes.com/2020/09/30/opinion/pre...,Our president as a terrible toddler.,When the nation looks back on the presidential...,A,27,2020-10-01T00:05:51+0000,article,...,Opinion,Op-Ed,925,Anyone Else Want to See Trump ‘Shut Up’?,Anyone Else Want Trump To ‘Shut Up’?,By Gail Collins,NaN,2020-10-01,00:05:51,Gail Collins
1,1,nyt://article/55c14af4-0542-53ef-80aa-65e2440e...,The Lakers fell into a deficit early but fough...,https://www.nytimes.com/2020/09/30/sports/bask...,The Lakers fell into a deficit early but fough...,LeBron James is not accustomed to Game 1s like...,NaN,NaN,2020-10-01T00:30:45+0000,article,...,Sports,News,1723,How the Lakers Beat the Heat in Game 1 of the ...,NaN,By Sopan Deb and Marc Stein,NaN,2020-10-01,00:30:45,"Sopan Deb, Marc Stein"
2,2,nyt://article/9a7ef9e0-1334-56b2-a7f1-288c4887...,"Instead of condemning violent groups, the pres...",https://www.nytimes.com/2020/09/30/opinion/tru...,"Instead of condemning violent groups, the pres...",President Trump didn’t hurt Joe Biden in Tuesd...,A,27,2020-10-01T00:43:28+0000,article,...,Opinion,Op-Ed,902,Trump Calls on Extremists to ‘Stand By’,Trump Calls On Extremists To ‘Stand By’,By Nicholas Kristof,NaN,2020-10-01,00:43:28,Nicholas Kristof


In [196]:
# ===================================
#   CREATE DataFrame 'article_author'
# ===================================

df_temp = df_Ar[['authors']].copy()

df_Ar_Au = df_temp.set_index(['authors'])

# explode: Transform each element of a list-like to a row, replicating index values.
df_Ar_Au = df_temp.apply(lambda x: x.str.split(',').explode())


##  CLEAN authors column

# Trim head and tail from white spaces
df_Ar_Au.authors = df_Ar_Au.authors.str.strip() 

# Let only one whitespace between words
df_Ar_Au.authors = df_Ar_Au.authors.replace(r'\s+', ' ', regex=True) 

# Index it, BUT save OLD index, that is the index of the article table!!!
df_Ar_Au = df_Ar_Au.reset_index()

# Replace where author is empty with a NaN
# and then drop the rows with NaN value
df_Ar_Au.authors.replace('', np.nan, inplace=True)
df_Ar_Au.dropna(subset=['authors'], inplace=True)


## RENAME columns
df_Ar_Au.rename(columns={'index': 'article_id', 'authors': 'author_name'}, inplace=True)

# find duplicates
# df_Ar_Au[df_Ar_Au.duplicated(['article_id', 'author_name'], keep = 'last')]

# Drop duplicates 
# because in the original set the authors are Greta Rybus, Galen Koch, Greta Rybus
# two time Greta Rybus
df_Ar_Au = df_Ar_Au.drop_duplicates()

print(df_Ar_Au.shape)

df_Ar_Au.head(5)

# Con NaN (158016, 1)
# Sin NaN (144898, 2)

(144895, 2)


,article_id,author_name
0,0,Gail Collins
1,1,Sopan Deb
2,1,Marc Stein
3,2,Nicholas Kristof
4,3,Mike Isaac


In [197]:
# ===================================
#   CREATE DataFrame 'author'
# ===================================

# Find unique authors  # Total 10446
unique_authors_list = df_Ar_Au.author_name.unique()

# Create DataFrame with Unique authors
df_Au = pd.DataFrame(unique_authors_list, columns =['author_name'])

# Reset index, BUT save old
df_Au = df_Au.reset_index()

# Rename columns
df_Au.rename(columns={'index': 'author_id'}, inplace=True)

print(df_Au.shape)
df_Au.head()

(10442, 2)


,author_id,author_name
0,0,Gail Collins
1,1,Sopan Deb
2,2,Marc Stein
3,3,Nicholas Kristof
4,4,Mike Isaac


In [198]:
# ===================================
#   MODIFY TABLE 'article_author'
# find the author_id from table 'author'
# ===================================

df_Ar_Au = df_Ar_Au.merge(df_Au, on=['author_name'])

# Drop column author_name
df_Ar_Au = df_Ar_Au.drop('author_name', axis=1)

df_Ar_Au.head()

,article_id,author_id
0,0,0
1,672,0
2,851,0
3,1222,0
4,1845,0


## ======== CREATE DATABASE ========

There is a tool to check the SQLite DB graphically <br />
<a href="https://sqlitestudio.pl/">https://sqlitestudio.pl/</a>

In [199]:
import os, sys
import sqlite3

In [200]:
# List databases created. Those are files
print(os.listdir('./nyt_sqlite_dbs/'))

['nyt_db - Kopie.db']


In [201]:
# Configure DB name to test
PATH_DB_TEST = './nyt_sqlite_dbs/'    #  <== You have to create this directory !!
DB_NAME_TEST = 'nyt_db.db'
print("Active DB name ==>", PATH_DB_TEST + DB_NAME_TEST)

Active DB name ==> ./nyt_sqlite_dbs/nyt_db.db


In [202]:
# !!!! IMPORTANT Always before closing the window !!!!
# cursor.close()
# conn.close()

In [203]:
# DROP DATABASE ==> Remove the file
# os.remove( PATH_DB_TEST + DB_NAME_TEST )

In [204]:
# ===================================
# CREATE DATABASE
# ===================================
conn = sqlite3.connect(PATH_DB_TEST + DB_NAME_TEST)

## ======== CREATE TABLES ========

In [205]:
# ===================================
# CREATE DB TABLE 'author'
# ===================================
conn.execute(
'''
CREATE TABLE IF NOT EXISTS author (
    author_id   INTEGER PRIMARY KEY AUTOINCREMENT,
    author_name TEXT UNIQUE
);
''')
conn.commit()

df_Au.to_sql('author', conn, if_exists='append', index = False)

10442

In [206]:
# ===================================
# CREATE DB TABLE 'article'
# ===================================

conn.execute(
'''
CREATE TABLE IF NOT EXISTS article (
    article_id       INTEGER UNIQUE PRIMARY KEY AUTOINCREMENT NOT NULL,
    original_id      VARCHAR,
    abstract         VARCHAR,
    web_url          VARCHAR,
    snippet          VARCHAR,
    lead_paragraph   VARCHAR,
    print_section    VARCHAR,
    print_page       VARCHAR,
    pub_date         VARCHAR,
    document_type    VARCHAR,
    news_desk        VARCHAR,
    section_name     VARCHAR,
    type_of_material VARCHAR,
    word_count       INTEGER DEFAULT 0,
    headline_main    VARCHAR,
    headline_print_headline VARCHAR,
    byline_original         VARCHAR,
    byline_organization     VARCHAR,
    a_date           VARCHAR,
    a_time           VARCHAR,
    authors          VARCHAR
);
''')
conn.commit()

df_Ar.to_sql('article', conn, if_exists='append', index = False)

126465

In [207]:
# ===================================
# CREATE DB TABLE 'article_author'
# ===================================
conn.execute(
'''
CREATE TABLE IF NOT EXISTS article_author (
    article_id        INTEGER NOT NULL,
    author_id         INTEGER NOT NULL,
    PRIMARY KEY ( article_id, author_id )
);
''')
conn.commit()

df_Ar_Au.to_sql('article_author', conn, if_exists='append', index = False)

144895

## ======== Check DB ========

In [208]:
# Check Created Tables
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('author',), ('sqlite_sequence',), ('article',), ('article_author',)]


In [209]:
# Check Table Schema
cursor = conn.cursor()
cursor.execute("PRAGMA table_info('author')").fetchall()

[(0, 'author_id', 'INTEGER', 0, None, 1),
 (1, 'author_name', 'TEXT', 0, None, 0)]

In [210]:
# Check Table Schema
cursor = conn.cursor()
cursor.execute("PRAGMA table_info('article_author')").fetchall()

[(0, 'article_id', 'INTEGER', 1, None, 1),
 (1, 'author_id', 'INTEGER', 1, None, 2)]

In [211]:
# Check Table Schema
cursor = conn.cursor()
cursor.execute("PRAGMA table_info('article')").fetchall()

[(0, 'article_id', 'INTEGER', 1, None, 1),
 (1, 'original_id', 'VARCHAR', 0, None, 0),
 (2, 'abstract', 'VARCHAR', 0, None, 0),
 (3, 'web_url', 'VARCHAR', 0, None, 0),
 (4, 'snippet', 'VARCHAR', 0, None, 0),
 (5, 'lead_paragraph', 'VARCHAR', 0, None, 0),
 (6, 'print_section', 'VARCHAR', 0, None, 0),
 (7, 'print_page', 'VARCHAR', 0, None, 0),
 (8, 'pub_date', 'VARCHAR', 0, None, 0),
 (9, 'document_type', 'VARCHAR', 0, None, 0),
 (10, 'news_desk', 'VARCHAR', 0, None, 0),
 (11, 'section_name', 'VARCHAR', 0, None, 0),
 (12, 'type_of_material', 'VARCHAR', 0, None, 0),
 (13, 'word_count', 'INTEGER', 0, '0', 0),
 (14, 'headline_main', 'VARCHAR', 0, None, 0),
 (15, 'headline_print_headline', 'VARCHAR', 0, None, 0),
 (16, 'byline_original', 'VARCHAR', 0, None, 0),
 (17, 'byline_organization', 'VARCHAR', 0, None, 0),
 (18, 'a_date', 'VARCHAR', 0, None, 0),
 (19, 'a_time', 'VARCHAR', 0, None, 0),
 (20, 'authors', 'VARCHAR', 0, None, 0)]

## ======== Simple QUERIES TEST DB ========

In [212]:
cursor = conn.execute('''  
SELECT * 
FROM article
LIMIT 2
''')

for row in cursor.fetchall():
    print (row)

(0, 'nyt://article/e467c2ae-2df3-5836-a6ca-b23d0d335e4f', 'Our president as a terrible toddler.', 'https://www.nytimes.com/2020/09/30/opinion/presidential-debate-trump.html', 'Our president as a terrible toddler.', 'When the nation looks back on the presidential debate we witnessed this week, do you think it’ll be remembered as:', 'A', '27', '2020-10-01T00:05:51+0000', 'article', 'OpEd', 'Opinion', 'Op-Ed', 925, 'Anyone Else Want to See Trump ‘Shut Up’?', 'Anyone Else Want Trump To ‘Shut Up’?', 'By Gail Collins', None, '2020-10-01', '00:05:51.000000', 'Gail Collins')
(1, 'nyt://article/55c14af4-0542-53ef-80aa-65e2440e69a8', 'The Lakers fell into a deficit early but fought back with a string of 3-pointers. The Heat were hobbled by injuries, a concern for the rest of the series.', 'https://www.nytimes.com/2020/09/30/sports/basketball/lakers-heat-live-game-1.html', 'The Lakers fell into a deficit early but fought back with a string of 3-pointers. The Heat were hobbled by injuries, a conce

In [213]:
cursor = conn.execute('''  
SELECT * 
FROM author
WHERE author_name LIKE '%Gail%'
LIMIT 5
''')

for row in cursor.fetchall():
    print (row)

(0, 'Gail Collins')
(2200, 'Gail Cornwall')
(3020, 'Abigail Pesta')
(4029, 'Abigail Tarttelin')
(4596, 'Abigail Stone')


In [214]:
## To test with DataFrame
# df_test = df3.loc[df3['authors'].str.contains("Collins", case=False)]
# df_test.head()

In [215]:
cursor = conn.execute('''  
SELECT * 
FROM article_author
LIMIT 5
''')

for row in cursor.fetchall():
    print (row)

(0, 0)
(672, 0)
(851, 0)
(1222, 0)
(1845, 0)


## ======== QUERY TEST ========

In [216]:
## Using WHERE clause
# Show the articles headlines with the word 'covid' 
# written by 'Gail Collins'

cursor = conn.execute('''  
SELECT ar.headline_main, au.author_name 
FROM article ar, author au, article_author arau
WHERE ar.article_id = arau.article_id
  AND arau.author_id = au.author_id
  AND au.author_name LIKE 'Gail Collins'
  AND ar.headline_main LIKE '%covid%'
LIMIT 20
''')

for row in cursor.fetchall():
    print (row)

('Covid Isn’t Finished Messing With Politics', 'Gail Collins')
('Covid 3.0, Biden 2.0 and Trump Number …', 'Gail Collins')


In [217]:
## Using JOINs
# Show the articles headlines with the word 'covid' 
# written by 'Gail Collins'

cursor = conn.execute('''  
SELECT ar.headline_main, au.author_name 
FROM article ar
  JOIN article_author arau ON ar.article_id  = arau.article_id
  JOIN author au           ON arau.author_id = au.author_id
WHERE au.author_name   LIKE 'Gail Collins'
  AND ar.headline_main LIKE '%covid%'
LIMIT 20
''')

for row in cursor.fetchall():
    print (row)

('Covid Isn’t Finished Messing With Politics', 'Gail Collins')
('Covid 3.0, Biden 2.0 and Trump Number …', 'Gail Collins')


In [218]:
## Using JOINs
# Show the articles word count total with the word 'covid' 
# written by 'Gail Collins'

cursor = conn.execute('''  
SELECT au.author_name, SUM(ar.word_count)
FROM article ar
  JOIN article_author arau ON ar.article_id  = arau.article_id
  JOIN author au           ON arau.author_id = au.author_id
WHERE au.author_name   LIKE 'Gail Collins'
  AND ar.headline_main LIKE '%covid%'
LIMIT 20
''')

for row in cursor.fetchall():
    print (row)

('Gail Collins', 3143)


In [219]:
## Using JOINs
# Show the nummer of articles per author
# who has written more articles about covid (covid in abstract)

cursor = conn.execute('''  
SELECT au.author_name, COUNT(ar.original_id) AS total_articles
FROM article ar
  JOIN article_author arau ON ar.article_id  = arau.article_id
  JOIN author au           ON arau.author_id = au.author_id
WHERE ar.headline_main LIKE '%covid%'
  AND au.author_name LIKE '%Gail%'
GROUP BY au.author_name
ORDER BY total_articles DESC, au.author_name ASC
LIMIT 20;
''')

for row in cursor.fetchall():
    print (row)


('Gail Collins', 2)
('Abigail A. Dumes', 1)


In [220]:
## Using JOINs
# Show the number of articles per section
# with more articles about covid (covid in abstract)

cursor = conn.execute('''  
SELECT ar.section_name, COUNT(ar.original_id) AS total_articles
FROM article ar
  JOIN article_author arau ON ar.article_id  = arau.article_id
  JOIN author au           ON arau.author_id = au.author_id
WHERE ar.abstract LIKE '%covid%'
GROUP BY ar.section_name
ORDER BY total_articles DESC, ar.section_name ASC
LIMIT 20;
''')

for row in cursor.fetchall():
    print (row)

('World', 522)
('U.S.', 438)
('Opinion', 229)
('Business Day', 171)
('Health', 168)
('Obituaries', 141)
('New York', 119)
('Briefing', 87)
('Well', 75)
('Arts', 67)
('Sports', 51)
('Podcasts', 49)
('Style', 31)
('Science', 29)
('Magazine', 23)
('Technology', 21)
('Books', 20)
('The Learning Network', 20)
('Travel', 19)
('Climate', 13)


## ======== INSERT  VALUE ========

In [221]:
# Variables from FastApi
abstract='DfasdfDDDDDDDDDDDDDDBBBBBabstract safadsaf asdfasdf asdfasdf'
section_name='Opinion'
headline_main='Breaking news headline TITULARRRR'
authors = 'Manuel Duque'

In [222]:
# ============================
# INSERT NEW on table 'article'
# ============================

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()

# YY-mm-dd H:M:S
a_date = now.strftime("%Y-%m-%d")
a_time = now.strftime("%H:%M:%S")

sql = '''  
INSERT OR IGNORE INTO article (abstract, section_name, headline_main, a_date, a_time, authors) 
VALUES ('{abstract}', '{section_name}', '{headline_main}', '{a_date}', '{a_time}', '{authors}');
'''

# sql
sql = sql.format(
        abstract=abstract, 
        section_name=section_name, 
        headline_main=headline_main, 
        a_date=a_date, 
        a_time=a_time, 
        authors=authors
)

# print(sql)

cursor = conn.execute(sql)

article_id = cursor.lastrowid
print("article_id last inserted in table 'article' is {}".format(last_article_id))

article_id last inserted in table 'article' is 126466


In [223]:
# ============================
# INSERT NEW on table 'author'
# ============================

author_id = ''

# Search author inserted
sql = ''' 
SELECT author_id 
FROM author 
WHERE author_name LIKE '{authors}';
'''
sql = sql.format(authors=authors)

cursor = conn.execute(sql)
records = cursor.fetchall()
row_count = len(records)

# If author does not exist
if row_count == 0:
    #print("Author is NOT in the table 'author'")
    sql = '''INSERT OR IGNORE INTO author (author_name)
             VALUES ('{authors}');
          '''
    sql = sql.format(authors=authors)
    print(sql)
    cursor = conn.execute(sql)
    author_id = cursor.lastrowid
    print("author_id last inserted in table 'author' is {}".format(author_id))
else:
    author_id = records[0][0]
    print(author_id)

print(author_id)

INSERT OR IGNORE INTO author (author_name)
             VALUES ('Manuel Duque');
          
author_id last inserted in table 'author' is 10442
10442


In [224]:
# ============================
# INSERT NEW on table 'article_author'
# ============================

# Search author inserted
sql = ''' 
SELECT article_id, author_id 
FROM article_author
WHERE article_id = {article_id}
  AND author_id = {author_id};
'''
sql = sql.format(article_id=article_id, author_id=author_id)

cursor = conn.execute(sql)
records = cursor.fetchall()
row_count = len(records)

# If author does not exist
if row_count == 0:
    #print("Author is NOT in the table 'author'")
    sql = '''INSERT OR IGNORE INTO article_author (article_id, author_id)
             VALUES ({article_id}, {author_id});
          '''
    sql = sql.format(article_id=article_id, author_id=author_id)
    print(sql)
    cursor = conn.execute(sql)
else:
    #author_id = records[0][0]
    print(records)

INSERT OR IGNORE INTO article_author (article_id, author_id)
             VALUES (126465, 10442);
          


In [225]:
conn.commit()

In [226]:
cursor = conn.execute(''' 
SELECT * FROM article WHERE authors = 'Manuel Duque';
''')
print(cursor)
for row in cursor.fetchall():
    print (row)

(126465, None, 'DfasdfDDDDDDDDDDDDDDBBBBBabstract safadsaf asdfasdf asdfasdf', None, None, None, None, None, None, None, None, 'Opinion', None, 0, 'Breaking news headline TITULARRRR', None, None, None, '2022-12-07', '02:25:56', 'Manuel Duque')


In [227]:
## Using JOINs
# Show the articles headlines with the word 'covid' 
# written by 'Manuel Duque'

cursor = conn.execute('''  
SELECT ar.article_id, ar.headline_main, au.author_id, au.author_name 
FROM article ar
  JOIN article_author arau ON ar.article_id  = arau.article_id
  JOIN author au           ON arau.author_id = au.author_id
WHERE au.author_name   LIKE 'Manuel Duque'
LIMIT 20
''')

for row in cursor.fetchall():
    print (row)

(126465, 'Breaking news headline TITULARRRR', 10442, 'Manuel Duque')


## ======== CLOSE SESSION ========

In [179]:
# !!!! IMPORTANT Always !!!!
# cursor.close()
# conn.close()

In [180]:
# DROP DATABASE ==> Remove the file
# os.remove( PATH_DB_TEST + DB_NAME_TEST )

In [ ]:
### COMMENT:
# I think it is better not to implement this 
# because it makes the things more complicated

# # CREATE TABLE section
# connection.execute(
# '''
# CREATE TABLE IF NOT EXISTS section (
#     id_section_name  INTEGER PRIMARY KEY AUTOINCREMENT,
#     section_name     TEXT
# );
# ''')

# # CREATE TABLE newsdesk
# connection.execute(
# '''
# CREATE TABLE IF NOT EXISTS newsdesk (
#     id_news_desk     INTEGER PRIMARY KEY AUTOINCREMENT,
#     news_desk_name   TEXT
# );
# ''')

## =====================================